In [28]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [29]:
#load the trainded model, scaler and encoder
model = load_model('model.h5')


# load the scaler and encoder
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

with open('onehot_encoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)


In [30]:
# Example data
input_data = {
    'CreditScore': 608,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 41,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [31]:
# One hot encode 'Geography'
geo_encoded= onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\Pranav\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [32]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,France,Male,41,3,60000,2,1,1,50000


In [33]:
# Encode categorical data
input_df['Gender'] = encoder.transform(input_df['Gender'])
input_df 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,France,1,41,3,60000,2,1,1,50000


In [34]:
# concatenation one hot encoded
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [35]:
input_df 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,1,41,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
# Scaling the input data
input_scaled=scaler.transform(input_df)

In [37]:
# predict churn
prediction=model.predict(input_scaled)

1/1 [==============================] - 0s 146ms/step


In [39]:
prediction_prob=prediction[0][0]
prediction_prob

0.05931357

In [40]:
if prediction_prob > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
